Installation of dependency

In [ ]:
!pip install geovoronoi[plotting]
!pip install geovoronoi descartes
!pip install geovoronoi descartes requests

This script generates and visualizes Voronoi regions within Germany by downloading Natural Earth country boundary data, generating random points within the country's boundaries, and computing Voronoi polygons clipped to the country's shape. It then calculates the area of each region and plots the results using GeoPandas and Matplotlib, saving the visualization as an image.

In [ ]:
import logging
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, Point
import requests
import tempfile
import os
import pandas as pd
from geovoronoi import coords_to_points, points_to_coords, voronoi_regions_from_coords, calculate_polygon_areas

# Setup logging
logging.basicConfig(level=logging.INFO)
geovoronoi_log = logging.getLogger('geovoronoi')
geovoronoi_log.setLevel(logging.INFO)
geovoronoi_log.propagate = True

# Parameters
N_POINTS = 35
COUNTRY = 'Germany'
np.random.seed(456)  # Different seed for variation

# Download Natural Earth data directly
print('Downloading Natural Earth data for country: %s' % COUNTRY)
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
temp_dir = tempfile.mkdtemp()
temp_zip = os.path.join(temp_dir, "ne_110m_admin_0_countries.zip")

# Download the data
response = requests.get(url)
with open(temp_zip, 'wb') as f:
    f.write(response.content)

# Read the shapefile from the downloaded zip
world = gpd.read_file(f"zip://{temp_zip}")
area = world[world.NAME == COUNTRY]  # Note: Using NAME instead of name for the Natural Earth data
assert len(area) == 1, f"Country {COUNTRY} not found in Natural Earth data"
print('CRS:', area.crs)  # gives epsg:4326 -> WGS 84

# Convert to Albers Equal Area CRS (ETRS89-extended / LAEA Europe)
area = area.to_crs(epsg=3035)
area_shape = area.iloc[0].geometry  # get the Polygon

# Generate random points within the bounds
minx, miny, maxx, maxy = area_shape.bounds
randx = np.random.uniform(minx, maxx, N_POINTS)
randy = np.random.uniform(miny, maxy, N_POINTS)
coords = np.vstack((randx, randy)).T

# Use only the points inside the geographic area
pts = [p for p in coords_to_points(coords) if p.within(area_shape)]  # converts to shapely Point
n_pts = len(pts)
print('Will use %d of %d randomly generated points that are inside geographic area' % (n_pts, N_POINTS))
coords = points_to_coords(pts)  # convert back to simple NumPy coordinate array
del pts

# Calculate the Voronoi regions, cut them with the geographic area shape and assign the points to them
region_polys, region_pts = voronoi_regions_from_coords(coords, area_shape)

# Calculate area in km²
poly_areas = calculate_polygon_areas(region_polys, m2_to_km2=True)  # converts m² to km²
print('Areas in km²:')
pprint(poly_areas)
print('Sum:')
print(sum(poly_areas.values()))

# Better approach using GeoDataFrames for plotting
def create_plot_using_geopandas(area_shape, region_polys, coords, region_pts, poly_areas):
    # Create GeoDataFrame for the country
    country_gdf = gpd.GeoDataFrame(geometry=[area_shape])

    # Create GeoDataFrame for Voronoi regions
    voronoi_data = []
    for region_id, region_poly in region_polys.items():
        area = poly_areas.get(region_id, 0)
        voronoi_data.append({
            'region_id': region_id,
            'area_label': f"{int(round(area))} km²",
            'geometry': region_poly
        })
    voronoi_gdf = gpd.GeoDataFrame(voronoi_data)

    # Create GeoDataFrame for points
    points_data = []
    for region_id, point_indices in region_pts.items():
        for idx in point_indices:
            x, y = coords[idx]
            points_data.append({
                'region_id': region_id,
                'geometry': Point(x, y)
            })
    points_gdf = gpd.GeoDataFrame(points_data)

    # Create the plot
    fig, ax = plt.subplots(figsize=(10, 10))

    # Plot base country
    country_gdf.plot(ax=ax, color='white', edgecolor='black')

    # Plot Voronoi regions
    voronoi_gdf.plot(ax=ax, column='region_id', cmap='Pastel1', alpha=0.5, edgecolor='black')

    # Plot points
    points_gdf.plot(ax=ax, color='black', markersize=20)

    # Add labels
    for _, row in voronoi_gdf.iterrows():
        centroid = row.geometry.centroid
        ax.text(centroid.x, centroid.y, row['area_label'],
                fontsize=7, ha='center', va='center', color='gray')

    # Set title and layout
    ax.set_title(f'{n_pts} random points and their Voronoi regions in {COUNTRY}\n')
    ax.set_aspect('equal')
    plt.tight_layout()

    return fig

# Generate and save the plot
fig = create_plot_using_geopandas(area_shape, region_polys, coords, region_pts, poly_areas)
plt.savefig('germany_voronoi_regions.png')
plt.show()

# Clean up temporary files
import shutil
shutil.rmtree(temp_dir)